In [36]:
import requests
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
import pandas as pd
import numpy as np
import re
import hgtk
import datetime
from time import sleep
import inspect
import sys
import logging
import schedule
import time
import os
params = {'host':'localhost', 'dbname':'KISA', 'user':'postgres', 'pwd':'dpafhqh1!'}
host = params['host']
dbname = params['dbname']
user = params['user']
pwd = params['pwd']
import psycopg2



In [100]:
customer_product_columns = ['sign_up_cd', 'customer_cd', 'product_cd', 'balance']
customer_columns = ['customer_cd', 'customer_nm',  'monthly_income', 'job', 
                                   'investment_propensity', 'recommendation']
income_expenditure_columns = ['income_expenditure_cd', 'customer_cd',  'month', 'monthly_income', 
                                   'monthly_expenditure', 'balance']
recommendation_columns = ['job', 'monthly_income', 'monthly_expenditure']
product_columns = ['product_cd', 'volatility', 'interest_rate']
expenditure_columns = ['payment_cd', 'month', 'customer_cd', 'expenditure']

# 테이블 만들기

### 고객 정보 DB

#### 테이블 명 : investmore.customer

In [76]:
conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
cur = conn.cursor()
cur.execute("""
CREATE TABLE investmore.customer(
    customer_cd character varying(100),
    customer_nm character varying(100),
    monthly_income character varying(100),
    job character varying(100),
    investment_propensity character varying(100),
    recommendation bigint
    )

TABLESPACE pg_default;""")

conn.commit()

### 고객 보유 상품 테이블

#### 테이블 명 : investmore.customer_product


In [62]:
conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
cur = conn.cursor()
cur.execute("""
CREATE TABLE investmore.customer_product(
    sign_up_cd character varying(100),
    customer_cd character varying(100),
    product_cd character varying(100),
    balance bigint
    )
TABLESPACE pg_default;""")

conn.commit()

### 고객 수입 - 지출 테이블
#### 테이블 명 : investmore.income_expenditure

In [82]:
conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
cur = conn.cursor()
cur.execute("""
CREATE TABLE investmore.income_expenditure(
    income_expenditure_cd character varying(100),
    customer_cd character varying(100),
    month bigint,
    monthly_income bigint,
    monthly_expenditure bigint,
    balance bigint
    )
TABLESPACE pg_default;""")

conn.commit()

### 상품 정보 DB

#### 테이블 명 : investmore.product

In [65]:
conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
cur = conn.cursor()
cur.execute("""
CREATE TABLE investmore.product(
    product_cd character varying(100),
    volatility character varying(100),
    interest_rate character varying(100)
    )
TABLESPACE pg_default;""")

conn.commit()

### 평균 지출 테이블

#### 테이블 명 : investmore.recommendation

In [134]:
conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
cur = conn.cursor()
cur.execute("""
CREATE TABLE investmore.recommendation(
    job character varying(100),    
    monthly_income bigint,
    monthly_expenditure bigint
    )

TABLESPACE pg_default;""")

conn.commit()

# 결재 내역 테이블
#### 테이블 명 : investmore.expenditure

In [105]:
conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
cur = conn.cursor()
cur.execute("""
CREATE TABLE investmore.expenditure(
    payment_cd character varying(100),    
    month bigint,
    customer_cd bigint,
    expenditure bigint
    )

TABLESPACE pg_default;""")

conn.commit()

# 데이터 가져오기



## 전체 데이터 가져오기


### 고객 정보 테이블

In [89]:
table_name = 'investmore.customer'

conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
cur = conn.cursor()

cur.execute("SELECT * FROM  "+table_name)
data = pd.DataFrame(cur.fetchall(), columns = customer_columns)
data

,customer_cd,customer_nm,monthly_income,job,investment_propensity,recommendation
0,1000000002,김탕진,2500000,직장인,저위험,1000000
1,1000000001,김소주,800000,대학생,고위험,500000


### 고객 수입 - 지출 테이블

In [91]:
table_name = 'investmore.income_expenditure'

conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
cur = conn.cursor()

cur.execute("SELECT * FROM  "+table_name)
data = pd.DataFrame(cur.fetchall(), columns = income_expenditure_columns)
data


,income_expenditure_cd,customer_cd,month,monthly_income,monthly_expenditure,balance
0,B00006,1000000002,6,2500000,2900000,-400000
1,B00005,1000000002,5,2500000,2800000,-300000
2,B00004,1000000002,4,2500000,3000000,-500000
3,B00003,1000000001,6,800000,800000,200000
4,B00002,1000000001,5,800000,700000,200000
5,B00001,1000000001,4,800000,600000,200000


### 상품 테이블

In [88]:
table_name = 'investmore.product'

conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
cur = conn.cursor()

cur.execute("SELECT * FROM  "+table_name)
data = pd.DataFrame(cur.fetchall(), columns = product_columns)
data

,product_cd,volatility,interest_rate
0,P00006,보통위험,1.568400
1,P00005,매우높은위험,6.651250
2,P00004,매우낮은위험,0.720000
3,P00003,다소높은위험,2.893556
4,P00002,높은위험,2.762569
5,P00001,낮은위험,0.952941


### 평균 지출 테이블

In [130]:
table_name = 'investmore.recommendation'

conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
cur = conn.cursor()

cur.execute("SELECT * FROM  "+table_name)
data = pd.DataFrame(cur.fetchall(), columns = recommendation_columns)
data

,job,monthly_income,monthly_expenditure
0,직장인,5100000,2500000
1,직장인,5000000,2460000
2,직장인,4900000,2370000
3,직장인,4800000,2280000
4,직장인,4700000,1350000
5,직장인,4600000,1450000
6,직장인,4500000,2080000
7,직장인,4400000,1920000
8,직장인,4300000,1750000
9,직장인,4200000,1980000


### 고객 보유 상품 테이블

In [93]:
table_name = 'investmore.customer_product'

conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
cur = conn.cursor()

cur.execute("SELECT * FROM  "+table_name)
data = pd.DataFrame(cur.fetchall(), columns = customer_product_columns)
data

,sign_up_cd,customer_cd,product_cd,balance
0,A00002,1000000002,1,2000000
1,A00001,1000000001,1,1000000


### 지출 내역 테이블

In [129]:
table_name = 'investmore.expenditure'

conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
cur = conn.cursor()

cur.execute("SELECT * FROM  "+table_name)
data = pd.DataFrame(cur.fetchall(), columns = expenditure_columns)
data

,payment_cd,month,customer_cd,expenditure
0,P00000000004,6,1000000002,800000
1,P00000000003,6,1000000002,600000
2,P00000000002,6,1000000001,400000
3,P00000000001,6,1000000001,200000


# 특정 데이터 가져오기

#### 고객DB`

In [57]:
table_name = 'investmore.customer'

conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
cur = conn.cursor()

cur.execute("SELECT * FROM  "+table_name + " WHERE customer_cd='10000000001'")

#### 상품DB

In [ ]:
table_name = 'investmore.product'

conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
cur = conn.cursor()

cur.execute("SELECT * FROM  "+table_name + " WHERE product_cd='10000000001'")



In [33]:
data_list = data.iloc[0].values.tolist()

In [34]:
customer_dict = dict()

for i in range(len(data_list)):
    customer_dict[data.columns[i]] = str(data_list[i]


In [35]:
customer_dict

{'customer_cd': '100485242323',
 'customer_nm': '홍길동',
 'product_cd': '00001',
 'balance': 3405040,
 'recent_monthly_payment': 1000000,
 'recent_monthly_income': 2000000,
 'proper_payment': 800000,
 'recent_monthly_accumulative_payment': 400000}

In [28]:
key = data.iloc[0]['customer_cd']
value = str(data.iloc[0].values.tolist())

customer_dict = {key:value}
# jsonify(customer_dict)
customer_dict

{'100485242323': "['100485242323', '홍길동', '00001', 3405040, 1000000, 2000000, 800000, 400000]"}

# 데이터 입력



In [70]:
def get_data_columns_and_data_sets(columns):
    columns

    a = '('

    for i in columns:
        a= a + str(i) + ','

    data_columns = a[:-1]+')'


    b = '('

    for i in columns:
        b = b + '%s' + ','

    data_sets= b[:-1]+')'
    return data_columns, data_sets



    

### 고객 보유 상품 테이블


In [72]:
# customer_product_columns = ['sign_up_cd', 'customer_cd', 'product_cd', 'balance']

sample_data = [['A00001', '1000000001', '1', '1000000'],['A00002', '1000000002', '1', '2000000']]

customer_product_df = pd.DataFrame(sample_data, columns = customer_product_columns)

data_columns, data_sets = get_data_columns_and_data_sets(customer_product_columns)


## 지정 필수!!!!
table_name = 'investmore.customer_product'
data = customer_product_df
##########

print(table_name +'꺼 넣기 시작 ' )
insert_data=  data.iloc[::-1].values.tolist()

for i in insert_data:
    try:
        i = [None if str(x) == str(np.nan) else x for x in i]
        cur.execute("INSERT INTO " + table_name + " " + data_columns + ' VALUES ' + data_sets , i) 
        print(str(i[0]))
        conn.commit()
    except Exception as e:
        print("*******Error********" + str(e))
        print("error data : " + str(i))
        conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
        cur = conn.cursor()
        pass    
conn.commit()
print(table_name + str('꺼끝'))

investmore.customer_product꺼 넣기 시작 
A00002
A00001
investmore.customer_product꺼끝


## 고객 정보 테이블


In [79]:
# customer_columns = ['customer_cd', 'customer_nm',  'monthly_income', 'job', 
#                                    'investment_propensity', 'recommendation']

sample_data = [['1000000001', '김소주', '800000', '대학생', '고위험', '500000'],
                         ['1000000002', '김탕진', '2500000', '직장인', '저위험', '1000000']]



data_columns, data_sets = get_data_columns_and_data_sets(customer_columns)


## 지정 필수!!!!
table_name = 'investmore.customer'
customer_df = pd.DataFrame(sample_data, columns = customer_columns)
data = customer_df
##########

print(table_name +'꺼 넣기 시작 ' )
insert_data=  data.iloc[::-1].values.tolist()

for i in insert_data:
    try:
        i = [None if str(x) == str(np.nan) else x for x in i]
        cur.execute("INSERT INTO " + table_name + " " + data_columns + ' VALUES ' + data_sets , i) 
        print(str(i[0]))
        conn.commit()
    except Exception as e:
        print("*******Error********" + str(e))
        print("error data : " + str(i))
        conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
        cur = conn.cursor()
        pass    
conn.commit()
print(table_name + str('꺼끝'))

investmore.customer꺼 넣기 시작 
1000000002
1000000001
investmore.customer꺼끝


## 고객 수입 - 지출 테이블

In [84]:
# income_expenditure_columns = ['income_expenditure_cd', 'customer_cd',  'month', 'monthly_income', 
#                                    'monthly_expenditure', 'balance']

sample_data = [['B00001', '1000000001', '4', '800000', '600000', '200000'],
                         ['B00002', '1000000001', '5', '800000', '700000', '200000'],
                         ['B00003', '1000000001', '6', '800000', '800000', '200000'],
                         ['B00004', '1000000002', '4', '2500000', '3000000', '-500000'],
                         ['B00005', '1000000002', '5', '2500000', '2800000', '-300000'],               
                         ['B00006', '1000000002', '6', '2500000', '2900000', '-400000']]


## 지정 필수!!!!
data_columns, data_sets = get_data_columns_and_data_sets(income_expenditure_columns)
table_name = 'investmore.income_expenditure'
income_expenditure_df = pd.DataFrame(sample_data, columns = income_expenditure_columns)
data = income_expenditure_df
##########

print(table_name +'꺼 넣기 시작 ' )
insert_data=  data.iloc[::-1].values.tolist()

for i in insert_data:
    try:
        i = [None if str(x) == str(np.nan) else x for x in i]
        cur.execute("INSERT INTO " + table_name + " " + data_columns + ' VALUES ' + data_sets , i) 
        print(str(i[0]))
        conn.commit()
    except Exception as e:
        print("*******Error********" + str(e))
        print("error data : " + str(i))
        conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
        cur = conn.cursor()
        pass    
conn.commit()
print(table_name + str('꺼끝'))







investmore.income_expenditure꺼 넣기 시작 
B00006
B00005
B00004
B00003
B00002
B00001
investmore.income_expenditure꺼끝


### 상품 테이블

In [85]:
# product_columns = ['product_cd', 'volatility', 'interest_rate']

sample_data = [['P00001', '낮은위험', '0.952941'], 
                         ['P00002', '높은위험', '2.762569'],
                         ['P00003', '다소높은위험', '2.893556'],
                         ['P00004', '매우낮은위험', '0.720000'],
                         ['P00005', '매우높은위험', '6.651250'],
                         ['P00006', '보통위험', '1.568400']]


## 지정 필수!!!!
data_columns, data_sets = get_data_columns_and_data_sets(product_columns)
table_name = 'investmore.product'
product_df = pd.DataFrame(sample_data, columns = product_columns)
data = product_df
##########

print(table_name +'꺼 넣기 시작 ' )
insert_data=  data.iloc[::-1].values.tolist()

for i in insert_data:
    try:
        i = [None if str(x) == str(np.nan) else x for x in i]
        cur.execute("INSERT INTO " + table_name + " " + data_columns + ' VALUES ' + data_sets , i) 
        print(str(i[0]))
        conn.commit()
    except Exception as e:
        print("*******Error********" + str(e))
        print("error data : " + str(i))
        conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
        cur = conn.cursor()
        pass    
conn.commit()
print(table_name + str('꺼끝'))



investmore.product꺼 넣기 시작 
P00006
P00005
P00004
P00003
P00002
P00001
investmore.product꺼끝


### 평균 지출 테이블

In [135]:
# recommendation_columns = ['job', 'monthly_income', 'monthly_expenditure']

table = pd.read_csv('income_pay_final.csv')
table['job'] = '직장인'
table['monthly_income'] = table['monthly_income'].apply(lambda x : x*10000)
table['monthly_expenditure'] = table['monthly_expenditure'].apply(lambda x : x*10000)

## 지정 필수!!!!
data_columns, data_sets = get_data_columns_and_data_sets(recommendation_columns)
table_name = 'investmore.recommendation'
recommendation_df = table[['job', 'monthly_income', 'monthly_expenditure']]
data = recommendation_df
##########

print(table_name +'꺼 넣기 시작 ' )
insert_data=  data.iloc[::-1].values.tolist()

for i in insert_data:
    try:
        i = [None if str(x) == str(np.nan) else x for x in i]
        cur.execute("INSERT INTO " + table_name + " " + data_columns + ' VALUES ' + data_sets , i) 
        print(str(i[0]))
        conn.commit()
    except Exception as e:
        print("*******Error********" + str(e))
        print("error data : " + str(i))
        conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
        cur = conn.cursor()
        pass    
conn.commit()
print(table_name + str('꺼끝'))




investmore.recommendation꺼 넣기 시작 
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
직장인
investmore.recommendation꺼끝


In [ ]:
P00000000001	6	10000000001	200000
P00000000002	6	10000000001	400000
P00000000003	6	10000000002	600000
P00000000004	6	10000000002	800000

In [106]:
sample_data = [['P00000000001', '6', '1000000001', '200000'], 
                         ['P00000000002', '6', '1000000001', '400000'],
                         ['P00000000003', '6', '1000000002', '600000'], 
                         ['P00000000004', '6', '1000000002', '800000']]


## 지정 필수!!!!
data_columns, data_sets = get_data_columns_and_data_sets(expenditure_columns)
table_name = 'investmore.expenditure'
expenditure_df = pd.DataFrame(sample_data, columns = expenditure_columns)
data = expenditure_df
##########

print(table_name +'꺼 넣기 시작 ' )
insert_data=  data.iloc[::-1].values.tolist()

for i in insert_data:
    try:
        i = [None if str(x) == str(np.nan) else x for x in i]
        cur.execute("INSERT INTO " + table_name + " " + data_columns + ' VALUES ' + data_sets , i) 
        print(str(i[0]))
        conn.commit()
    except Exception as e:
        print("*******Error********" + str(e))
        print("error data : " + str(i))
        conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
        cur = conn.cursor()
        pass    
conn.commit()
print(table_name + str('꺼끝'))





investmore.expenditure꺼 넣기 시작 
P00000000004
P00000000003
P00000000002
P00000000001
investmore.expenditure꺼끝


In [14]:
sample_data = [['100485242323', '홍길동', '00001', '3405040', '1000000', '2000000', '800000', '400000']]

In [41]:
customer_df = pd.read_csv('sample_data.csv')
customer_df.columns = customer_columns
customer_df

,customer_cd,customer_nm,product_cd,balance,previous_monthly_payment,previous_monthly_income,proper_payment,recent_monthly_accumulative_payment,exceed
0,10000000001,김지연,1,2000000,800000,1300000,650000,980000,-330000
1,10000000002,오은택,2,4000000,700000,2000000,800000,720000,80000
2,10000000003,노우제,3,7000000,1500000,3410000,1200000,2000000,-800000
3,10000000004,안상선,4,9000000,2000000,4500000,2000000,2500000,-500000


In [46]:
## 지정 필수!!!!
table_name = 'investmore.customer'
data = customer_df
print(table_name +'꺼 넣기 시작 ' )
insert_data=  data.iloc[::-1].values.tolist()

for i in insert_data:
    try:
        i = [None if str(x) == str(np.nan) else x for x in i]
        cur.execute("INSERT INTO " + table_name + " " + data_columns + ' VALUES ' + data_sets , i) 
        print(str(i[0]))
        conn.commit()
    except Exception as e:
        print("*******Error********" + str(e))
        print("error data : " + str(i))
        conn = psycopg2.connect('host={0} dbname={1} user={2} password={3}'.format(host, dbname, user, pwd))
        cur = conn.cursor()
        pass    
conn.commit()
print(table_name + str('꺼끝'))

investmore.customer꺼 넣기 시작 
10000000004
10000000003
10000000002
10000000001
investmore.customer꺼끝
